In [35]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [36]:
!pip install timedelta

In [37]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-04 22:56:38.105797


In [38]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [39]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [40]:
current_date.to_datetime64()

numpy.datetime64('2025-03-04T22:00:00.000000000')

In [41]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [42]:
fetch_data_to

Timestamp('2025-03-04 22:00:00+0000', tz='UTC')

In [43]:
fetch_data_from

Timestamp('2025-02-03 22:00:00+0000', tz='UTC')

In [12]:
"""
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    
    # Convert string inputs to datetime if necessary
    from_date = pd.to_datetime(from_date)
    to_date = pd.to_datetime(to_date)

    # Ensure timezone consistency (convert naive timestamps to UTC)
    if from_date.tz is None:
        from_date = from_date.tz_localize("UTC")
    else:
        from_date = from_date.tz_convert("UTC")

    if to_date.tz is None:
        to_date = to_date.tz_localize("UTC")
    else:
        to_date = to_date.tz_convert("UTC")

    # Validate input dates
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    # Ensure 'pickup_datetime' is timezone-aware (convert to UTC if necessary)
    rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'])
    if rides['pickup_datetime'].dt.tz is None:
        rides['pickup_datetime'] = rides['pickup_datetime'].dt.tz_localize("UTC")
    else:
        rides['pickup_datetime'] = rides['pickup_datetime'].dt.tz_convert("UTC")

    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides
"""

'\nfrom datetime import datetime, timedelta\nfrom typing import Union\nimport pandas as pd\nfrom src.data_utils import load_and_process_taxi_data\n\ndef fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:\n    \n    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).\n\n    Args:\n        from_date (datetime or str): The start date for the data batch.\n        to_date (datetime or str): The end date for the data batch.\n\n    Returns:\n        pd.DataFrame: A DataFrame containing the simulated production data.\n    \n    # Convert string inputs to datetime if necessary\n    from_date = pd.to_datetime(from_date)\n    to_date = pd.to_datetime(to_date)\n\n    # Ensure timezone consistency (convert naive timestamps to UTC)\n    if from_date.tz is None:\n        from_date = from_date.tz_localize("UTC")\n    else:\n        from_date = from_date.tz_convert("UTC")\n\n    if to_date.tz is None:\n        to_date

In [44]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # ✅ Ensure datetime is timezone-naive
    historical_from_date = pd.to_datetime(historical_from_date).tz_localize(None)
    historical_to_date = pd.to_datetime(historical_to_date).tz_localize(None)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    
    # ✅ Ensure `pickup_datetime` is also timezone-naive
    rides_from['pickup_datetime'] = pd.to_datetime(rides_from['pickup_datetime']).dt.tz_localize(None)

    # ✅ Apply filtering correctly
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to['pickup_datetime'] = pd.to_datetime(rides_to['pickup_datetime']).dt.tz_localize(None)
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [45]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [47]:
rides

,pickup_datetime,pickup_location_id
816246,2025-02-12 16:25:44,2
1122947,2025-02-15 16:56:40,2
6662,2025-02-04 04:14:50,3
8263,2025-02-04 06:08:40,3
76016,2025-02-04 18:17:57,3
...,...,...
2943320,2025-03-04 21:56:22,263
2940886,2025-03-04 21:56:50,263
2942068,2025-03-04 21:57:22,263
2942407,2025-03-04 21:58:49,263


In [48]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [49]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-03 22:00:00,2,0
1,2025-02-03 23:00:00,2,0
2,2025-02-04 00:00:00,2,0
3,2025-02-04 01:00:00,2,0
4,2025-02-04 02:00:00,2,0
...,...,...,...
174691,2025-03-04 17:00:00,263,117
174692,2025-03-04 18:00:00,263,132
174693,2025-03-04 19:00:00,263,99
174694,2025-03-04 20:00:00,263,86


In [50]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174696 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174696 non-null  datetime64[ns]
 1   pickup_location_id  174696 non-null  int16         
 2   rides               174696 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [51]:
"""
import pandas as pd
import numpy as np
import pytz

# Convert pickup_hour to EST
ts_data["pickup_hour_est"] = ts_data["pickup_hour"].dt.tz_localize("UTC").dt.tz_convert("America/New_York")

# Load the taxi zone lookup CSV file to map location IDs to names
lookup_csv_path = "/Users/sharmilamanoj/Downloads/taxi_zone_lookup.csv"
location_lookup = pd.read_csv(lookup_csv_path)

# Rename the column for merging consistency
location_lookup.rename(columns={"LocationID": "pickup_location_id", "Zone": "pickup_location_name"}, inplace=True)

# Identify top 10 pickup locations by frequency
top_10_locations = ts_data["pickup_location_id"].value_counts().nlargest(10).index

# Merge ts_data with the lookup table to get names for the top 10 locations
ts_data = ts_data.merge(location_lookup[["pickup_location_id", "pickup_location_name"]], on="pickup_location_id", how="left")

# Keep names only for the top 10 locations, others remain NaN
ts_data.loc[~ts_data["pickup_location_id"].isin(top_10_locations), "pickup_location_name"] = None

# Apply FFT to rides data (grouped by pickup location)
ts_data["fft_real"] = 0.0
ts_data["fft_imag"] = 0.0

for location in ts_data["pickup_location_id"].unique():
    rides_series = ts_data.loc[ts_data["pickup_location_id"] == location, "rides"].values
    fft_values = np.fft.fft(rides_series)
    ts_data.loc[ts_data["pickup_location_id"] == location, "fft_real"] = np.real(fft_values)
    ts_data.loc[ts_data["pickup_location_id"] == location, "fft_imag"] = np.imag(fft_values)

# Display the updated DataFrame
print(ts_data.head())  # Print the first few rows
ts_data.to_csv("enhanced_ts_data.csv", index=False)  # Save the file for review
"""


'\nimport pandas as pd\nimport numpy as np\nimport pytz\n\n# Convert pickup_hour to EST\nts_data["pickup_hour_est"] = ts_data["pickup_hour"].dt.tz_localize("UTC").dt.tz_convert("America/New_York")\n\n# Load the taxi zone lookup CSV file to map location IDs to names\nlookup_csv_path = "/Users/sharmilamanoj/Downloads/taxi_zone_lookup.csv"\nlocation_lookup = pd.read_csv(lookup_csv_path)\n\n# Rename the column for merging consistency\nlocation_lookup.rename(columns={"LocationID": "pickup_location_id", "Zone": "pickup_location_name"}, inplace=True)\n\n# Identify top 10 pickup locations by frequency\ntop_10_locations = ts_data["pickup_location_id"].value_counts().nlargest(10).index\n\n# Merge ts_data with the lookup table to get names for the top 10 locations\nts_data = ts_data.merge(location_lookup[["pickup_location_id", "pickup_location_name"]], on="pickup_location_id", how="left")\n\n# Keep names only for the top 10 locations, others remain NaN\nts_data.loc[~ts_data["pickup_location_id"].

In [52]:
"""
for location in top_10_locations:
    rides_series = ts_data.loc[ts_data["pickup_location_id"] == location, "rides"].values
    
    if len(rides_series) > 1:  # Ensure there is more than 1 data point
        fft_values = np.fft.fft(rides_series)
        ts_data.loc[ts_data["pickup_location_id"] == location, "fft_real"] = np.real(fft_values)
        ts_data.loc[ts_data["pickup_location_id"] == location, "fft_imag"] = np.imag(fft_values)
    else:
        ts_data.loc[ts_data["pickup_location_id"] == location, ["fft_real", "fft_imag"]] = np.nan
"""

'\nfor location in top_10_locations:\n    rides_series = ts_data.loc[ts_data["pickup_location_id"] == location, "rides"].values\n    \n    if len(rides_series) > 1:  # Ensure there is more than 1 data point\n        fft_values = np.fft.fft(rides_series)\n        ts_data.loc[ts_data["pickup_location_id"] == location, "fft_real"] = np.real(fft_values)\n        ts_data.loc[ts_data["pickup_location_id"] == location, "fft_imag"] = np.imag(fft_values)\n    else:\n        ts_data.loc[ts_data["pickup_location_id"] == location, ["fft_real", "fft_imag"]] = np.nan\n'

In [53]:
#print(ts_data[["pickup_location_id", "fft_real", "fft_imag"]].head(10))


In [54]:
#print(ts_data["pickup_location_id"].unique())


In [55]:
##getting top 10 locations
#top_10_locations = ts_data["pickup_location_id"].value_counts().nlargest(10).index
#print(top_10_locations)


In [56]:
#print(ts_data[ts_data["pickup_location_id"] == 2].head(10))


In [57]:
# print(ts_data["pickup_location_name"].value_counts())


In [58]:
os.environ["HOPSWORKS_API_KEY"] = "jaThBXjYztKj0Mr2.jvcnZsSjp04pK6c4uuSoK7LtNHJOe9EFRZtegX2KXmxrHAv6MGSPiIY6iODgbe8S"
api_key = os.getenv("HOPSWORKS_API_KEY")

import os
from dotenv import load_dotenv

# Load .env file
load_dotenv("hopsworks.env")  # Make sure this file exists

# Print the API key (for debugging)
api_key = os.getenv("HOPSWORKS_API_KEY")
if api_key:
    print("✅ API Key Loaded Successfully!")
else:
    print("❌ API Key Not Found. Check .env file.")

import os
import hopsworks
api_key = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login(api_key_value=api_key)
print("Logged into Hopsworks:", project.name)

feature_store = project.get_feature_store()

FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

feature_group = feature_store.get_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
)

✅ API Key Loaded Successfully!
2025-03-04 18:00:26,363 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:00:26,370 INFO: Initializing external client
2025-03-04 18:00:26,371 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:00:27,020 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Logged into Hopsworks: nyc_taxi_proj


In [27]:
""""
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)
"""


'"\nimport hopsworks\n\n# connect to the project\nproject = hopsworks.login(\n    project=config.HOPSWORKS_PROJECT_NAME,\n    api_key_value=config.HOPSWORKS_API_KEY\n)\n\n# connect to the feature store\nfeature_store = project.get_feature_store()\n\n# connect to the feature group\nfeature_group = feature_store.get_feature_group(\n    name=config.FEATURE_GROUP_NAME,\n    version=config.FEATURE_GROUP_VERSION,\n)\n'

In [59]:
print(ts_data.columns)


Index(['pickup_hour', 'pickup_location_id', 'rides'], dtype='object')


In [60]:
print(feature_group.schema)


[Feature('pickup_hour', 'timestamp', None, True, False, None, None, 1402010), Feature('pickup_location_id', 'int', None, True, False, None, None, 1402010), Feature('rides', 'int', None, False, False, None, None, 1402010)]


In [61]:
# from hsfs.feature import Feature  # Import Feature before using it

# feature_store = project.get_feature_store()

# # Define the updated feature group with the new schema
# new_feature_group = feature_store.create_feature_group(
#     name="new_fft_ts_data",
#     version=1,  # Update as needed
#     description="Updated feature group with FFT features and pickup location names",
#     primary_key=["pickup_hour", "pickup_location_id"],
#     features=[
#         Feature("pickup_hour", "timestamp"),
#         Feature("pickup_location_id", "int"),
#         Feature("rides", "int"),
#         Feature("pickup_hour_est", "timestamp"),
#         Feature("pickup_location_name", "string"),
#         Feature("fft_real", "double"),
#         Feature("fft_imag", "double"),
#     ]
# )

# # Insert the enhanced dataset
# new_feature_group.insert(ts_data, write_options={"wait_for_job": False})


In [62]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 174696/174696 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214665/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)